<h1>Street view time machine-03.05.2021</h1>
<h2>Jason Gravel, PhD.<br>Department of Criminal Justice<br>Temple University</h2><br>
<a href="https://wwww.jasongravel.com">Personal website</a><br>
<a href="https://twitter.com/GravelJ">Twitter</a>


This code was used to generate historical panoramic views for specific addresses in Philadelphia before and after an intervention seeking to clean up or remediate housing units. <br> <br>
The code is adapted from code posted online by different python programmers, most notably: <br>
<li>robolyst (<a href="https://github.com/robolyst/streetview">https://github.com/robolyst/streetview</a>)</li>
<li>cplusx (<a href="https://github.com/cplusx/google-street-view-panorama-download">https://github.com/cplusx/google-street-view-panorama-download</a>)</li>


In [1]:
#Packages needed
import pandas as pd
import geocoder
import streetview
import itertools
import matplotlib.pyplot as plt
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import urllib, json, os

You will need a Google developper account to activate your access to the API.<br>

To setup account:https://cloud.google.com/maps-platform/<br>

NOTE: Even though it tells you to setup a billing account, I have never had to pay for its usage. 
That said, make sure you understand what is defined as a "request" to the API, and whether your usage
would go over the limit. Because I never got anywhere close to the limit, the code below is not at all
optimized to limit the number of requests made to the API, i.e. I get a lot of information I don't actually
end up using. If you have a lot of requests to make, that something to keep in mind.


In [2]:
#Copy your API key below
API_KEY="API KEY"

In [16]:
#FUNCTIONS 

#This function basically takes a dictionary with the lat, longs and 
#returns all panoids close to the submitted coordinates.
def getPanoIds(geodct):
    """
     This part of the code uses the package streetview created by
     robolyst (https://github.com/robolyst/streetview)
     The function returns all panorama ids (panoids) in the vicinity of
     a given lat,lon coordinate. Google's streetview
     API only returns the most recent pictures, but this code provides
     a work around that allows us to get panoramas taken in the past.

     For unknown reasons, some panoids 
     have dates attached to them, while others don't.

     Note that this package is not maintained by Google, and thus
     may no longer function in the future if Google changes their API.
    """
    panoids={}
    for k,v in geodct.items():
        panoids[k]=streetview.panoids(lat=v['lat'], lon=v['lon'])
    return panoids

#This code was adapted from the code provided by cplusx
#https://github.com/cplusx/google-street-view-panorama-download
#The function download_panorama_v3 takes a panoid and retrieves 
#tiles from google streetview then stitches them into a 2d panoramic
#picture
def tiles_info(panoid, zoom=5):
    """
    Generate a list of a panorama's tiles and their position.
    The format is (x, y, filename, fileurl)
    """
#     image_url = 'http://maps.google.com/cbk?output=tile&panoid={}&zoom={}&x={}&y={}'
    image_url = "http://cbk0.google.com/cbk?output=tile&panoid={}&zoom={}&x={}&y={}"

    # The tiles positions
    coord = list(itertools.product(range(26), range(13)))

    tiles = [(x, y, "%s_%dx%d.jpg" % (panoid, x, y), image_url.format(panoid, zoom, x, y)) for x, y in coord]

    return tiles
def download_panorama_v3(panoid, zoom=5, disp=False):
    '''
    v3: save image information in a buffer. (v2: save image to dist then read)
    input:
        panoid: which is an id of image on google maps
        zoom: larger number -> higher resolution, from 1 to 5, better less than 3, some location will fail when zoom larger than 3
        disp: verbose of downloading progress, basically you don't need it
    output:
        panorama image (uncropped)
    '''
    tile_width = 512
    tile_height = 512
    # img_w, img_h = int(np.ceil(416*(2**zoom)/tile_width)*tile_width), int(np.ceil(416*( 2**(zoom-1) )/tile_width)*tile_width)
    img_w, img_h = 416*(2**zoom), 416*( 2**(zoom-1) )
    tiles = tiles_info( panoid, zoom=zoom)
    valid_tiles = []
    # function of download_tiles
    for i, tile in enumerate(tiles):
        x, y, fname, url = tile
        if disp and i % 20 == 0:
            print("Image %d / %d" % (i, len(tiles)))
        if x*tile_width < img_w and y*tile_height < img_h: # tile is valid
            # Try to download the image file
            while True:
                try:
                    response = requests.get(url, stream=True)
                    break
                except requests.ConnectionError:
                    print("Connection error. Trying again in 2 seconds.")
                    time.sleep(2)
            valid_tiles.append( Image.open(BytesIO(response.content)) )
            del response
            
    # function to stich
    panorama = Image.new('RGB', (img_w, img_h))
    i = 0
    for x, y, fname, url in tiles:
        if x*tile_width < img_w and y*tile_height < img_h: # tile is valid
            tile = valid_tiles[i]
            i+=1
            panorama.paste(im=tile, box=(x*tile_width, y*tile_height))
    return np.array(panorama)



def GetStreet(Add,SaveLoc,ID,API_KEY,size=(600,640),fov=50,heading=None, pitch=0, use_pano=False):
    
    #cleans parameters
    key="&key=" + API_KEY
    size=str(size[0])+"x"+str(size[1])
    fov=str(fov)
    if heading is None:
        heading=""
    else:
        heading=str(heading)
    pitch=str(pitch)
    if use_pano:
        selectstr="pano="
    else:
        selectstr="location="
        
    #gets image metadata
    base2="https://maps.googleapis.com/maps/api/streetview/metadata?"+selectstr
    MyUrl2 = base2 + urllib.parse.quote_plus(Add) + key
    r=urllib.request.urlopen(MyUrl2)
    data = json.loads(r.read().decode('utf-8'))
    if data['status']=='OK':
    
        #retrieves the image
        base = "https://maps.googleapis.com/maps/api/streetview?size=%s&fov=%s&heading=%s&pitch=%s&%s"%(size,fov,heading,pitch,selectstr)
        MyUrl = base + urllib.parse.quote_plus(Add) + key #added url encoding
        fi = ID+'_'+data['date'] + ".jpg"
        urllib.request.urlretrieve(MyUrl, os.path.join(SaveLoc,fi))
    else:
        print('Check %s: Streetview error code %s'%(str(k),data['status']))
    


<h1>Geocoding addresses using google API</h1>

You can try to use data that's already been geocoded elsewhere. I'm assuming that as long as it uses 
Google Maps' standard projection and the coordinates are in lat, long, it should work. But because of 
the weird procedures involved with identifying the panoids taken at different points in time (below),
it is probably safer to just start with a clean geocoding of your addresses using google maps and the
package geocoder.

In [5]:

#CSV file--My file had a column ADDRESS with a string of the address to be geocoded, and some unique ID (ai)
df=pd.read_csv('yourfile.csv')

#basically I create a dictionary keyed by my location ID with the street address as a value
attr=list(zip(df.ADDRESS.tolist(),df.aid.tolist()))
attrdct={aid:{'address':a} for (a,aid) in attr}
#Don't forget to add the city, state, country, if your addresses don't have it!
addr={k:v['address']+', Philadelphia, Pennsylvania, United States' for (k,v) in attrdct.items()}


In [10]:
#Iterate over the items in the address dictionary and returns the response from the Google maps API.
#Geodct stores all the fields returned by the API--lots of it is likely to be unnecessary so that's a way
#to reduce the number of requests you make to the API (i think?)

#check stores the addresses that were not found by google. Note that if you enter 
#a street address that doesn't exist, Google is pretty robust and will get you the closest address. 
#In some cases though, it will return a street corner or another location, which may not be necessarily
#close to where you intend to go. Check the "accuracy" parameter in the results. For example, if you're
#looking for precise buildings, you should get "ROOFTOP" as a result of accuracy. RANGE_INTERPOLATED, 
#GEOMETRIC_CENTER, and APPROXIMATE are of suspicious quality 
geodct0={}
check={}
for k,v in test.items():
    g=geocoder.google(v, key=API_KEY)
    if g.ok:
        geodct0[k]=g.json
        geodct0[k]['orig']=v
    else:
        check[k]=v
check2=[]
for k,v in geodct0.items():
    if v['accuracy']!='ROOFTOP':
        check2.append(k)
if len(check)==0:
    print('All addresses were geocoded')
else:
    print('%d addresses failed to geocode. Check the following ids:'%len(check))
    
        
    for e,x in enumerate(check):
        if e<10:
            print('Check address id '+str(x))
        elif e==10:
            print('%d additional addresses to check. print(check) to see full list.'%(len(check)-10))
        else:
            break
            
if len(check2)==0:
    print('Geocoded addresses appear to be accurate')
else:
    print('%d addresses were geocoded but are suspicious. Check the following ids:'%len(check2))
    
        
    for e,x in enumerate(check2):
        if e<10:
            print('Check address id '+str(x))
        elif e==10:
            print('%d additional addresses to check. print(check2) to see full list.'%(len(check2)-10))
        else:
            break

#This just cleans geodct to keep only the information you need for the other parts of the code
geodct={k:{'lat':v['lat'],'lon':v['lng'],'google_address':v['address']} for k,v in geodct0.items()}


All addresses were geocoded
Geocoded addresses appear to be accurate


<h1>Getting historical panorama pictures</h1>

In [13]:
#Run the function to get the pano ids
panoids=getPanoIds(geodct)
#Cleans the panoids, and keeps only the ones with a year attached to them
years=[]
panosyr0={}
for k,v in panoids.items():
    panosyr0[k]=[]
    for x in v:
        if 'year' in x.keys():
            panosyr0[k].append(x)
            years.append(x['year'])
panosyr={}
for k,v in panosyr0.items():
    panosyr[k]={x:[] for x in range(min(years),max(years)+1)}
    for x in v:
        panosyr[k][x['year']].append(x)
#Set the folder to save the pictures
myloc='streetview_test/'
#This will iterate over all the years available and generate a panoramic picture
#It will save the file as ID_Year_Month_pano.jpg
#You can adjust the parameter zoom to get higher quality pictures. 5 is the max I think, but 
#using it can sometimes lead to issues (or so says the internet).
cnt=0
for k,v in panosyr.items():
    for yr,val in v.items():
        if yr>2018:
            for x in val:
                try:
                    panorama = download_panorama_v3(x['panoid'], zoom=3, disp=False)
                    plt.imsave(myloc+k+'_'+str(yr)+'_'+str(x['month'])+'_pano.jpg', panorama)
                    cnt+=1
                except:
                    pass
print('Downloaded %d panorama pictures in folder %s'%(cnt,myloc))

Downloaded 3 panorama pictures in folder streetview_test/


<h1>Getting regular pictures from street view</h1>
Since you are getting the historical panoid stored in panosyr in the above steps, you can pass those ids to this code in order to get flat pictures. I think there is a way to do so within the streetview package as well. 

In [17]:
#set folder to save images
myloc="streetview_test/"
check=[]
for k,v in geodct.items():
    
    try:
        GetStreet(Add=v['google_address'],#needs to be a string with the street address, a string with lat,lng, or a string of the panoid
                  SaveLoc=myloc,#path to save files
                  ID=k,#location id
                  API_KEY=API_KEY,
                  size=(600,640),#See below for definitions of parameters
                  fov=50,
                  heading=None,
                  pitch=0,
                  use_pano=False)#if using panoid, set this to True
    except:
        check.append(k)
        pass

<h1>Street view static API parameters</h1>
<a href="https://developers.google.com/maps/documentation/streetview/overview">https://developers.google.com/maps/documentation/streetview/overview</a>

Starts with: https://maps.googleapis.com/maps/api/streetview?{parameters} <br>
Each parameter is seperated by "&"<br>


Required parameters:<br>
<li>Either one of:</li>
    <ul><li>location: text string of a place or street address of a lat/lng value</li></ul>
    <ul><li>pano: a specific panorama ID</li></ul>

<li>key: your API key</li>

<li>size: the output size of the image in pixels. Size is specified as {width}x{height} - for example, size=600x400 returns an image 600 pixels wide, and 400 high.</li>

Optional parameters:<br>
<li>heading: compass heading of the camera (0 to 360). 0 and 360=North, 90=East, 180=South, 270=West. When not included, it defaults to facing the location given (e.g. facing the building with the address given).
<li>fov: the horizontal field of view of the image. The field of view is expressed in degrees, with a maximum allowed value of 120. When dealing with a fixed-size viewport, as with a Street View image of a set size, field of view in essence represents zoom, with smaller numbers indicating a higher level of zoom. (default is 90)
<li>pitch:(default is 0) specifies the up or down angle of the camera relative to the Street View vehicle. This is often, but not always, flat horizontal. Positive values angle the camera up (with 90 degrees indicating straight up); negative values angle the camera down (with -90 indicating straight down).
<li>radius (default is 50) sets a radius, specified in meters, in which to search for a panorama, centered on the given latitude and longitude. Valid values are non-negative integers.

Example:
https://maps.googleapis.com/maps/api/streetview?size=600x300&location=46.414382,10.013988&fov=90&heading=151.78&pitch=-0.76&key=YOUR_API_KEY